In [265]:
import sys
# sys.path.append('/mnt/6088FEB688FE89B4/Users/arsen/Desktop/code/git/python-utils-ak')
sys.path.append(r'C:\Users\Mi\Desktop\code\git\python-utils-ak')
from utils_ak.reflexion import load as _load

def _import_globals(module):
    for x in dir(module):
        if not x.startswith('_'):
            globals()[x] = getattr(module, x)
            
def load(module_obj, module_name=None, reload=False, import_globals=False):
    module = _load(module_obj, module_name, reload, import_globals=False)
    if import_globals:
        _import_globals(module)
    return module

load('src/time.py', import_globals=True, reload=True)
load('src/block.py', import_globals=True, reload=True)
load('src/color.py', import_globals=True, reload=True)
load('src/drawing.py', import_globals=True, reload=True)
load('src/interval.py', import_globals=True, reload=True)

<module 'src/interval.py' from 'src/interval.py'>

# Generate boilings

In [266]:
import math
import pandas as pd

from utils_ak.numeric import custom_round

In [224]:
import sys
# sys.path.append('/mnt/6088FEB688FE89B4/Users/arsen/Desktop/code/git/2020.10-umalat/umalat')
sys.path.append(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat')
from app.models import *
from app.models import Boiling as BoilingModel
from app.models import Termizator as TermizatorModel

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# sqlite_filepath = '/home/akadaner/Desktop/data.sqlite'
sqlite_filepath = r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite"
engine = create_engine(f"sqlite:///{sqlite_filepath}")
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [4]:
all_skus = session.query(SKU).all()
df_skus = pd.DataFrame(all_skus, columns=['sku'])
df_skus['name'] = df_skus['sku'].apply(lambda sku:sku.name)
df_skus['percent'] = df_skus['sku'].apply(lambda sku:sku.boiling.percent)

In [5]:
df_skus[df_skus['percent'] == 2.7]

,sku,name,percent
0,<SKU 1>,"Моцарелла ""Pretto"" (для бутербродов), 45%, 0,2...",2.7
1,<SKU 2>,"Моцарелла ""Pretto"", 45%, 1,2 кг, в/у",2.7
2,<SKU 3>,"Моцарелла ""Unagrande"", 45%, 0,12 кг, ф/п (кубики)",2.7
3,<SKU 4>,"Моцарелла ""Unagrande"", 45%, 3 кг, пл/л",2.7
4,<SKU 5>,"Моцарелла (палочки), 45%, кг, пл/л",2.7
5,<SKU 6>,"Моцарелла без лактозы для сэндвичей ""Unagrande...",2.7
12,<SKU 13>,"Моцарелла для пиццы ""Pretto"", 45 %, 0,46 кг, т...",2.7
13,<SKU 14>,"Моцарелла для пиццы ""Unagrande"", 45%, 0,46 кг,...",2.7
14,<SKU 15>,"Моцарелла для пиццы ""Фермерская коллекция"", 45...",2.7
15,<SKU 16>,"Моцарелла для пиццы «Fine Life», 45%, 0,37 кг,...",2.7


In [6]:
session.query(SKU).first().name

'Моцарелла "Pretto" (для бутербродов), 45%, 0,2 кг, т/ф, (9 шт)'

In [7]:
request = {'Моцарелла Фиор ди Латте в воде "Красная птица", 45%, 0,125 кг, ф/п': 1, 
          'Моцарелла Фиор Ди Латте в воде "Pretto", 45%, 0,125 кг, ф/п, (8 шт)': 1}

df = pd.DataFrame(request.items(), columns=['name', 'qty'])
termizator = session.query(Termizator).first()
df['sku'] = df['name'].apply(lambda s: session.query(SKU).filter(SKU.name == s).first())
sku = df.iloc[0]['sku']
df['boiling'] = df['sku'].apply(lambda sku: session.query(BoilingModel).filter(BoilingModel.id == sku.boiling_id).first())
df

,name,qty,sku,boiling
0,"Моцарелла Фиор ди Латте в воде ""Красная птица""...",1,<SKU 27>,<Boiling 6>
1,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, ...",1,<SKU 24>,<Boiling 6>


In [8]:
BOILING_CONF = df.iloc[0]['boiling']
termizator.pouring_time = 30

In [267]:
def make_water_melting_and_packing(line, boiling_conf):
    maker = BlockMaker(default_push_func=dummy_push)
    make = maker.make
    
    with make('melting_and_packing'):
        packing_t_time = 0

        timings = []
        timings.append(boiling_conf.meltings.serving_time)
        # todo: specify number of kg 
        # todo: fix factor 1.3
        # melting time
        timings.append(custom_round(850 / boiling_conf.meltings.speed * 60 * 1.3, 5, rounding='ceil'))

        total_time_size = sum(timings)
        packing_t_time += timings[0]
        packing_t_time += boiling_conf.meltings.first_cooling_time
        packing_t_time += boiling_conf.meltings.second_cooling_time

        with make('melting', y=10, time_size=total_time_size):
            with make(y=0):
                make('melting_label', time_size=4 * 5)
                make('melting_name', time_size=total_time_size - 4 * 5, name='чильеджина 5кг/сердечки/фдл 0,125(безлактозная)/0,1')
            with make(y=1):
                make('serving', time_size=timings[0])
                make('melting_process', time_size=timings[1], speed=900)
            with make(y=2):
                make(time_size=timings[0], visible=False)
                make('cooling1', time_size=boiling_conf.meltings.first_cooling_time)
                make('cooling2', time_size=boiling_conf.meltings.second_cooling_time)

        timings = []
        timings.append(custom_round(850 / boiling_conf.meltings.speed * 60 * 1.3, 5, rounding='ceil'))
        total_time_size = sum(timings)

        with make('packing', t=packing_t_time / 5, time_size=total_time_size, push_func=add_push):
            with make(y=0):
                make('packing_label', time_size=3 * 5)
                make('packing_name', time_size=total_time_size - 3 * 5, name='чильеджина 5кг/сердечки/фдл 0,125(безлактозная)/0,1')
            with make(y=1):
                make('packing_brand', time_size=total_time_size, name='безлактозная/претто')

                    # todo: add configurations
        #         with make(y=2):
        #             make('configuration', size=3, visible=False)
        #             make('configuration', size=1)
        #             make('configuration', size=5, visible=False)
        #             make('configuration', size=1)
    
    res = maker.root.children[0]
    res.rel_props['size'] = max(c.end for c in res.children)
    return res

make_water_melting_and_packing('1', BOILING_CONF)

melting_and_packing (0, 36]
  melting (0, 21]
    block (0, 0]
      melting_label (0, 4]
      melting_name (4, 21]
    block (0, 0]
      serving (0, 6]
      melting_process (6, 21]
    block (0, 0]
      block (0, 6]
      cooling1 (6, 11]
      cooling2 (11, 21]
  packing (21.0, 36.0]
    block (0, 0]
      packing_label (0, 3]
      packing_name (3, 15]
    block (0, 0]
      packing_brand (0, 15]

In [268]:
def make_boiling(line, boiling_conf, block_num, boiling_type='water'):
    maker = BlockMaker(default_push_func=dummy_push)
    make = maker.make

    with make('boiling', block_num=block_num, line=line, boiling_type=boiling_type):
        # todo: make boiling size

        timings = []
        timings.append(boiling_conf.pourings.pouring_time)
        timings.append(boiling_conf.pourings.soldification_time)
        timings.append(boiling_conf.pourings.cutting_time)
        timings.append(boiling_conf.pourings.pouring_off_time)
        timings.append(boiling_conf.pourings.extra_time)
        
        with make('pouring', y=cast_row(f'water_{line}'), time_size=sum(timings)):
            with make(y=0):
                make('termizator', time_size=termizator.pouring_time)
                make('pouring_name', time_size=sum(timings) - termizator.pouring_time, name='чильеджина  3,6 альче  8500кг')
            with make(y=1):
                make('pouring_and_fermenting', time_size=timings[0])
                make('soldification', time_size=timings[1])
                make('cutting', time_size=timings[2])
                make('pouring_off', time_size=timings[3])
                make('extra', time_size=timings[4])

        # todo: specify parameter
        make('drenator', size=cast_t('04:00'), visible=False)
    
        if boiling_type == 'water':
            make(make_water_melting_and_packing(line, boiling_conf))        
        elif boiling_type == 'salt':
            raise Exception('Not supported')

    res = maker.root.children[0]
    res.rel_props['size'] = max(c.end for c in res.children)

    return res

make_boiling(1, BOILING_CONF, 12)

boiling (0, 110]
  pouring (0, 26]
    block (0, 0]
      termizator (0, 6]
      pouring_name (6, 26]
    block (0, 0]
      pouring_and_fermenting (0, 8]
      soldification (8, 14]
      cutting (14, 21]
      pouring_off (21, 24]
      extra (24, 26]
  drenator (26, 74]
  melting_and_packing (74, 110]
    melting (0, 21]
      block (0, 0]
        melting_label (0, 4]
        melting_name (4, 21]
      block (0, 0]
        serving (0, 6]
        melting_process (6, 21]
      block (0, 0]
        block (0, 6]
        cooling1 (6, 11]
        cooling2 (11, 21]
    packing (21.0, 36.0]
      block (0, 0]
        packing_label (0, 3]
        packing_name (3, 15]
      block (0, 0]
        packing_brand (0, 15]

In [272]:
# ROWS = {'water_0': 6, 'water_1': 9, 'salt_0': 15, 'salt_1': 18, 'water_melting': 24, 'water_packing': 29, 'salt_melting': 33, 'salt_packing': 46}
ROWS = {'water_0': 1, 'water_1': 3, 'water_melting': 5, 'water_packing': 8}

def cast_row(obj):
    if isinstance(obj, int):
        return obj
    elif isinstance(obj, str):
        return ROWS[obj]
    else:
        raise Exception('Unknown object type')
        
def boiling_validator(parent, boiling):
    if not parent.children:
        return
    b1 = max(parent.children, key=lambda b: b.rel_props.get('t', 0))
    b2 = boiling
    
    b1.rel_props['props_mode'] = 'absolute'
    b2.rel_props['props_mode'] = 'absolute'
    
    b1.upd_abs_props()
    b2.upd_abs_props()
    

    validate_disjoint(b1['pouring'][0]['termizator'], b2['pouring'][0]['termizator'])
    if b1.props['line'] == b2.props['line']:
        validate_disjoint(b1['pouring'], b2['pouring'])
    
    if b1.props['boiling_type'] == b2.props['boiling_type'] == 'water':
        validate_disjoint(b1['melting_and_packing']['melting'], b2['melting_and_packing']['melting'])
    elif b1.props['boiling_type'] == b2.props['boiling_type'] == 'salt':
        raise Exception('Not impelmented')
    else:
        pass
    
    if b1.props['boiling_type'] == b2.props['boiling_type']:
        validate_disjoint(b1['melting_and_packing']['packing'], b2['melting_and_packing']['packing'])
        
    b1.rel_props.pop('props_mode')
    b2.rel_props.pop('props_mode')
    
    b1.upd_abs_props()
    b2.upd_abs_props()
    
    

In [273]:
root = Block('root', t=cast_t('01:45'))
boiling_1 = make_boiling(line='0', boiling_conf=BOILING_CONF, block_num=12)
boiling_2 = make_boiling(line='1', boiling_conf=BOILING_CONF, block_num=13)
dummy_push(root, boiling_1)
dummy_push(root, boiling_2, beg='last_beg', max_tries=200, validator=boiling_validator)
root

root (21, 21]
  boiling (0, 110]
    pouring (0, 26]
      block (0, 0]
        termizator (0, 6]
        pouring_name (6, 26]
      block (0, 0]
        pouring_and_fermenting (0, 8]
        soldification (8, 14]
        cutting (14, 21]
        pouring_off (21, 24]
        extra (24, 26]
    drenator (26, 74]
    melting_and_packing (74, 110]
      melting (74, 95]
        block (0, 0]
          melting_label (0, 4]
          melting_name (4, 21]
        block (0, 0]
          serving (0, 6]
          melting_process (6, 21]
        block (0, 0]
          block (0, 6]
          cooling1 (6, 11]
          cooling2 (11, 21]
      packing (95.0, 110.0]
        block (0, 0]
          packing_label (0, 3]
          packing_name (3, 15]
        block (0, 0]
          packing_brand (0, 15]
  boiling (21, 131]
    pouring (21, 47]
      block (21, 21]
        termizator (21, 27]
        pouring_name (6, 26]
      block (0, 0]
        pouring_and_fermenting (0, 8]
        soldification (8, 14

In [274]:
root

root (21, 21]
  boiling (0, 110]
    pouring (0, 26]
      block (0, 0]
        termizator (0, 6]
        pouring_name (6, 26]
      block (0, 0]
        pouring_and_fermenting (0, 8]
        soldification (8, 14]
        cutting (14, 21]
        pouring_off (21, 24]
        extra (24, 26]
    drenator (26, 74]
    melting_and_packing (74, 110]
      melting (74, 95]
        block (0, 0]
          melting_label (0, 4]
          melting_name (4, 21]
        block (0, 0]
          serving (0, 6]
          melting_process (6, 21]
        block (0, 0]
          block (0, 6]
          cooling1 (6, 11]
          cooling2 (11, 21]
      packing (95.0, 110.0]
        block (0, 0]
          packing_label (0, 3]
          packing_name (3, 15]
        block (0, 0]
          packing_brand (0, 15]
  boiling (21, 131]
    pouring (21, 47]
      block (21, 21]
        termizator (21, 27]
        pouring_name (6, 26]
      block (0, 0]
        pouring_and_fermenting (0, 8]
        soldification (8, 14

In [275]:
STYLE = {
    'pouring': {'beg_time': '01:00'},
    'termizator': {'text': '{block_num} налив'},
    'pouring_name': {'text': '{name}'},
    'pouring_and_fermenting': {'text': 'налив/внесение закваски', 'color': 'orange'},
    'soldification': {'text': 'схватка', 'color': 'yellow'},
    'cutting': {'text': 'резка/обсушка', 'color': '#92d050'},
    'pouring_off': {'text': 'слив', 'color': 'red'},
    'melting': {'beg_time': '07:00', 'y': cast_row('water_melting')},
    'melting_label': {'text': '{block_num}'},
    'melting_name': {'text': '{name}'},
    'serving': {'color': 'orange', 'text': 'подача и вымешивание'},
    'melting_process': {'color': 'orange', 'text': 'плавление {speed} кг/ч'},
    'cooling1': {'text': 'охлаж1 <{size} * 5> мин'},
    'cooling2': {'text': 'охлаж2 <{size} * 5> мин'},
    'packing': {'beg_time': '07:00', 'y': cast_row('water_packing')},
    'packing_label': {'text': '{block_num}'},
    'packing_name': {'text': '{name}'},
    'packing_brand': {'color': '#f2dcdb', 'text': '{name}'},
    'configuration': {'color': 'red'}
}

# Final Water Algo

In [87]:
root = Block('root', t=cast_t('01:45'))

line = 0
for i, row in df.iterrows():
    boiling = make_boiling(line=line, boiling_conf=row['boiling'], block_num=12)
    dummy_push(root, boiling, beg='last_beg', max_tries=200, validator=boiling_validator)
    line = (line + 1) % 2

In [276]:
draw_schedule(root, STYLE, 'output.xlsx', init_sheet)

termizator 1 (14,20]
pouring_name 1 (20,40]
pouring_and_fermenting 2 (14,22]
soldification 2 (22,28]
cutting 2 (28,35]
pouring_off 2 (35,38]
extra 2 (38,40]
melting_label 5 (16,20]
melting_name 5 (20,37]
serving 6 (16,22]
melting_process 6 (22,37]
cooling1 7 (22,27]
cooling2 7 (27,37]
packing_label 8 (37.0,40.0]
packing_name 8 (40.0,52.0]
packing_brand 9 (37.0,52.0]
termizator 3 (35,41]
pouring_name 3 (41,61]
pouring_and_fermenting 4 (35,43]
soldification 4 (43,49]
cutting 4 (49,56]
pouring_off 4 (56,59]
extra 4 (59,61]
melting_label 5 (37,41]
melting_name 5 (41,58]
serving 6 (37,43]
melting_process 6 (43,58]
cooling1 7 (43,48]
cooling2 7 (48,58]
packing_label 8 (58.0,61.0]
packing_name 8 (61.0,73.0]
packing_brand 9 (58.0,73.0]


# Salt boiling

In [78]:
block_num = 12
line = '1'
boiling_conf = df_skus[df_skus['percent'] == 2.7].iloc[0]['sku'].boiling